In [17]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing   
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

In [18]:
TestFILE='Vanessa_Dumont_redo41921_2021-04-12_myeng110120_predictoutcome.csv'
out='Vanessa_DuMont_Rredo41921.xlsx'

In [19]:
cleanedF = os.path.join("C:/Users/jpcor/GoogleDrive/Data Science/Clauset's/PCR Analysis/CleanedData/", 'Combined_2017-19-20_RawData2020-11-01_myeng110120_outcome_clean.csv')#'Combined_2017-19-20_RawData2020-10-26_myeng092620_short.csv'
PCR_Data_M = pd.read_csv(cleanedF)

In [20]:
TestF = os.path.join("C:/Users/jpcor/GoogleDrive/Data Science/Clauset's/PCR Analysis/CleanedData/TestData/", TestFILE)
Test_Data_M = pd.read_csv(TestF)

In [21]:
PCR_Data_M

,outcome,outcome_clean,temp1,melting_temp2,annealing_temp2,extension_temp2,cycles2,temp3,band_size,dna_amt,...,Homodimer_maxTm,GC_ClampMin,GC_ClampMax,MaxTm_IDT,MinTm_IDT,MaxGC,MinGC,MaxPlen,MinPlen,TotPrimer
0,0,0,94,94,60,72,44,72,369.0,0.0,...,0.000000,35,39,58.394749,55.977067,82.352941,50.000000,24,17,0.4
1,0,0,94,94,60,72,44,72,369.0,0.0,...,0.000000,35,36,55.977067,52.726430,55.000000,50.000000,24,20,0.4
2,0,0,94,94,60,72,44,72,369.0,0.0,...,0.000000,35,39,59.131380,55.977067,70.000000,50.000000,24,20,0.4
3,1,1,94,94,60,72,44,72,369.0,0.0,...,12.001510,39,39,58.394749,55.507339,82.352941,57.142857,21,17,0.4
4,1,1,94,94,60,72,44,72,369.0,0.0,...,12.001510,36,39,55.507339,52.726430,57.142857,55.000000,21,20,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,1,1,95,95,55,72,35,72,188.0,1000.0,...,7.984100,22,27,71.670175,60.697236,48.000000,47.826087,46,25,20.0
286,1,0,95,95,60,72,36,72,502.0,1000.0,...,14.014201,28,33,73.138543,61.899024,52.380952,44.827586,42,29,20.0
287,1,1,95,95,60,72,36,72,417.0,1000.0,...,31.259604,31,31,74.118054,61.117186,57.894737,42.857143,38,28,20.0
288,1,0,95,95,60,72,36,72,427.0,1000.0,...,36.755084,34,41,72.642995,63.627070,51.162791,46.428571,43,28,20.0


In [22]:
initialRemove=['outcome','outcome_clean','Tm_p1_p3orig', 'Tm_p2_p3orig', 'Tm_p1_p3','Tm_p2_p3','Tm_p1_BSU','Tm_p2_BSU','temp1','MaxPlen','MinPlen']
initialRemove2=['Tm_p1_p3orig', 'Tm_p2_p3orig', 'Tm_p1_p3', 'Tm_p2_p3','Tm_p1_BSU', 'Tm_p2_BSU','temp1','MaxPlen','MinPlen']

In [23]:
set(initialRemove)-set(PCR_Data_M.columns.values)

set()

In [24]:
PCR_Data=PCR_Data_M.drop(initialRemove,axis=1)
Test_Data=Test_Data_M.drop(initialRemove2,axis=1)
def name(df,df_norm):
    names={}
    for i in range(len(df.columns.values)):
        names[i]=df.columns.values[i]
    df_norm.rename(columns=names,inplace=True)
    return df_norm
#Normalize PCR_Data and normalize test PCR_Data
#normalize columns
feat = PCR_Data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()#create scaler object
feat_scaled = min_max_scaler.fit_transform(feat)#scale each feature between 0 and 1
PCR_Data_norm = pd.DataFrame(feat_scaled)#create PCR_Dataframe from normalized PCR_Data
PCR_Data_norm = name(PCR_Data,PCR_Data_norm)

#Fit predict data
feat_pred = Test_Data.values
pred_scaled = min_max_scaler.transform(feat_pred)
Test_norm = pd.DataFrame(pred_scaled)
Test_norm = name(Test_Data,Test_norm)

In [25]:
finalremove=['outcome_clean','HPin_Tm_Max','t1(sec)','annealing_t3(sec)','MinGC','Heterodimer_tm','Tm_dif_IDT','MaxGC','extension_t4(sec)']
rfc = RandomForestClassifier(n_estimators=1000,oob_score=True,class_weight="balanced",max_features='auto',n_jobs=-1,random_state=42)
PCR_Data_norm['outcome_clean']=PCR_Data_M['outcome_clean']
PCR_Data_norm_t = shuffle(PCR_Data_norm,random_state=42)
X = PCR_Data_norm_t.drop(finalremove,axis=1) #(axis zero index 1 column)
y = PCR_Data_norm_t['outcome_clean']

rfc.fit(X,y)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=-1,
                       oob_score=True, random_state=42)

In [26]:
finalremove_p=['HPin_Tm_Max','t1(sec)','annealing_t3(sec)','MinGC','Heterodimer_tm','Tm_dif_IDT','MaxGC','extension_t4(sec)']

X_pred=Test_norm.drop(finalremove_p,axis=1)
outcome=rfc.predict(X_pred)

In [27]:
outcome

array([0, 0, 0], dtype=int64)

In [28]:
#Export outcomes to excel file with important features
lbs1=pd.read_csv('FeatureRelabel.csv')
lbs1[lbs1['Old Title']=='Hpin_Tm_Max']='HPin_Tm_Max'
lbs=dict(zip(lbs1['Old Title'],lbs1['New Title']))
s=['t1(sec)', 'annealing_t3(sec)', 'MinGC', 'Heterodimer_tm', 'Tm_dif_IDT', 'MaxGC', 'extension_t4(sec)']
mapper={}
for l in lbs.keys():
    if l not in s:
        mapper[l]=lbs[l]
Test_Data.rename(mapper=mapper,axis=1,inplace=True)
Test_Data.drop(finalremove_p,axis=1)
Test_Data['MODEL_PREDICTION']=outcome
Test_Data[['MODEL_PREDICTION','Melting Cycle Temp.', 'Annealing Cycle Temp', 'Extension  Temp',
       'Amplification Cycles', 'Final Extension Cycle Temp.',
       'Product Band Size', 'Template Amount', 't1(sec)',
       'Melting Cycle Time', 'annealing_t3(sec)', 'extension_t4(sec)',
       'Final Extension Time', 'Tm_dif_IDT', 'HPin_Tm_Max',
       'Heterodimer_tm', 'Max Homodimer Tm', 'Min GC Clamp Strength',
       'Max GC Clamp Strength', 'Max Primer Tm', 'Min Primer Tm', 'MaxGC',
       'MinGC', 'Total Primer']].to_excel(out,index=False)

In [29]:
Test_Data.columns.values

array(['Melting Cycle Temp.', 'Annealing Cycle Temp', 'Extension  Temp',
       'Amplification Cycles', 'Final Extension Cycle Temp.',
       'Product Band Size', 'Template Amount', 't1(sec)',
       'Melting Cycle Time', 'annealing_t3(sec)', 'extension_t4(sec)',
       'Final Extension Time', 'Tm_dif_IDT', 'HPin_Tm_Max',
       'Heterodimer_tm', 'Max Homodimer Tm', 'Min GC Clamp Strength',
       'Max GC Clamp Strength', 'Max Primer Tm', 'Min Primer Tm', 'MaxGC',
       'MinGC', 'Total Primer', 'MODEL_PREDICTION'], dtype=object)

In [30]:
labels=initialRemove
labels.append('outcome')
labels.append('outcome_clean')